# Setup

In [ ]:
from fastai.vision.all import *

# Data Loaders

In [ ]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                  get_items=get_image_files,
                  get_y=parent_label,
                  item_tfms=Resize(128),
                  splitter=RandomSplitter(seed=42),
                  batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders("train/", bs=bs)

In [ ]:
dls = get_dls(64, 224)

In [ ]:
dls.vocab

In [ ]:
dls.train.show_batch(max_n=8, nrows=2)

# Training

### Extra

In [ ]:
dls = get_dls(128, 64)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=LabelSmoothingCrossEntropy(), metrics=accuracy, cbs=MixUp()).to_fp16()
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr)
learn.recorder.plot_loss()
learn.save("freeze1")

In [ ]:
learn.load("freeze1")
learn.dls = get_dls(64, 128)
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, lr)
learn.recorder.plot_loss()
learn.save("freeze2")

In [ ]:
learn.load("freeze2")
learn.unfreeze()
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(4, lr_max=lr)
learn.recorder.plot_loss()
learn.save("unfreeze")

### Actual

In [ ]:
learn = cnn_learner(get_dls(64, 128), resnet50, metrics=error_rate)
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr_max=lr)
learn.recorder.plot_loss()

In [ ]:
learn.unfreeze()
lr = learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, lr_max=lr)
learn.recorder.plot_loss()

# Evaluation

In [ ]:
preds,targs = learn.tta()
accuracy(preds, targs).item() #0.4199095070362091

In [ ]:
learn.export("export.pkl")